In [1]:
import os
import json

In [3]:
archivos = []
for root, subdirs, files in os.walk('../data/06_models'):
    if len(files) <= 1:
        continue

    da_technique, model = root.split('/')[-2:]
    dict_data = {
        'da_technique': da_technique,
        'model': model
    }

    with open(root + '/metrics.json', 'r') as f:
        metrics = json.load(f)
    dict_data.update(metrics)

    with open(root + '/confusion_matrix.txt', 'r') as f:
        confusion_matrix = f.read()
    metrics_confusion_matrix = confusion_matrix.split('\n')[:3]
    metrics_confusion_matrix = {record.split(':')[0].replace(' ', '_'): float(record.split(':')[1]) for record in metrics_confusion_matrix}
    dict_data.update(metrics_confusion_matrix)

    archivos.append(dict_data)

In [23]:
import pandas as pd
pd.set_option("display.precision", 4)

In [24]:
df = pd.DataFrame(archivos)
df

,da_technique,model,test_acc,test_loss,test_f1,test_drift_score,test_drift_pvalue,test_a_distance,global_correct,mean_correct,mean_IoU
0,target_only,lenet,58.1814,1.3711,0.5675,0.0666,0.0,1.9494,58.2,57.9,44.0
1,mdd,resnet,98.6437,0.0498,0.9863,0.0615,0.0,1.8987,98.6,98.6,97.3
2,mdd,lenet,98.5569,0.0456,0.9854,0.0399,0.0,1.7468,98.6,98.6,97.1
3,source_only,resnet,98.8498,0.0388,0.9883,0.0632,0.0,1.9306,98.8,98.8,97.7
4,source_only,lenet,98.1120,0.0611,0.9810,0.0469,0.0,1.9515,98.1,98.1,96.3
5,dann,resnet,98.9041,0.0377,0.9890,0.1379,0.0,1.9776,98.9,98.9,97.8
6,dann,lenet,98.2205,0.0593,0.9821,0.0090,0.0,1.6774,98.2,98.2,96.5
7,afn,resnet,98.2856,0.1308,0.9827,0.0040,0.0,1.0886,98.3,98.3,96.6
8,afn,lenet,98.6220,0.0662,0.9859,0.0117,0.0,1.5747,98.6,98.6,97.3
9,adda,lenet,91.9054,0.2406,0.9184,0.0399,0.0,1.8495,91.9,91.8,85.2


In [25]:
df = (
    pd.DataFrame(archivos)
    .assign(
        test_acc=lambda x: x.test_acc / 100
    )
    .assign(
        da_technique=lambda x: x.da_technique.map({
            'source_only': 'SIN AD',
            'target_only': 'TARGET',
            'mdd': 'MDD',
            'dann': 'DANN',
            'afn': 'AFN',
            'adda': 'ADDA',
        }),
        model=lambda x: x.model.map({
            'resnet': 'ResNet',
            'lenet': 'LeNet',
        })
    )
    .rename(columns={
        'da_technique': "T.A.D",
        'model': "Modelo",
        "test_acc": "Accuracy",
        "test_f1": "F1",
        "test_drift_score": "MMD",
        "test_a_distance": "Dist. A",
    })
    .filter([
        "T.A.D",
        "Modelo",
        "Accuracy",
        "F1",
        "MMD",
        "Dist. A"
    ])
)

In [26]:
print(df.to_latex(index=False))

\begin{tabular}{llrrrr}
\toprule
 T.A.D & Modelo &  Accuracy &     F1 &    MMD &  Dist. A \\
\midrule
TARGET &  LeNet &    0.5818 & 0.5675 & 0.0666 &   1.9494 \\
   MDD & ResNet &    0.9864 & 0.9863 & 0.0615 &   1.8987 \\
   MDD &  LeNet &    0.9856 & 0.9854 & 0.0399 &   1.7468 \\
SIN AD & ResNet &    0.9885 & 0.9883 & 0.0632 &   1.9306 \\
SIN AD &  LeNet &    0.9811 & 0.9810 & 0.0469 &   1.9515 \\
  DANN & ResNet &    0.9890 & 0.9890 & 0.1379 &   1.9776 \\
  DANN &  LeNet &    0.9822 & 0.9821 & 0.0090 &   1.6774 \\
   AFN & ResNet &    0.9829 & 0.9827 & 0.0040 &   1.0886 \\
   AFN &  LeNet &    0.9862 & 0.9859 & 0.0117 &   1.5747 \\
  ADDA &  LeNet &    0.9191 & 0.9184 & 0.0399 &   1.8495 \\
\bottomrule
\end{tabular}



/var/folders/7d/x4wlq5h90vs57r1y1x58xn540000gn/T/ipykernel_2087/1842088038.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(index=False))
